# 2 - Construindo um LLM do Zero: Tokenização, um mal necessário

Este é o **segundo** de uma série de oito artigos que podem ser encontrados no meu medium. Acesse o primeiro artigo da série aqui: [Construindo um LLM: entendendo os Grandes Modelos de Linguagem](https://blog.zfab.me/construindo-um-llm-entendendo-os-grandes-modelos-de-linguagem-b37884219eaa)

--

Redes neurais profundas, e LLMs são redes neurais profundas, não podem processar texto diretamente e por isso precisamos transforma-lo em algo que o modelo possa consumir e aprender É nesse momento que entra uma etapa crucial na criação de um modelo de linguagem: a tokenização.

Imagine que você está tentando montar um quebra-cabeça sem saber como a imagem final deveria ser. As peças estão lá, mas sem entender o contexto ou a forma como se encaixam, é difícil formar uma imagem coerente. Da mesma forma, um modelo de linguagem precisa que o texto seja dividido em "peças" menores, ou seja, tokens, para que ele possa começar a construir uma representação significativa do conteúdo.

In [10]:
import re
import os
import sentencepiece as spm
import polars as pl

## Preparando o texto para o modelo

A criação de um LLM (Large Language Model ou Modelo de Linguagem de Grande Escala) começa pela preparação cuidadosa do texto que será utilizado no treinamento. Esse passo é fundamental para garantir que o modelo seja capaz de compreender o conteúdo, capturar o significado das palavras e reconhecer suas relações semânticas e sintáticas. Para alcançar esse objetivo, todo o conteúdo de treinamento precisa ser convertido em números - um formato que o modelo pode processar e, a partir disso, aprender de forma eficiente.

A abordagem mais simples e intuitiva para processar texto é atribuir um número único a cada palavra e substituir as palavras por esses identificadores sempre que aparecerem. Por exemplo, uma frase como "Eu adoro ciência de Dados!" poderia ser convertida em "1, 4, 10, 2, 13, 25", onde o número 1 representa a palavra "eu", o 13 corresponde a "dados" e o 25 indica o ponto de exclamação.
Porém para chegarmos nessa lista de palavras e identificadores (o chamado vocabulário) precisamos decidir como dividir nosso texto para identificação das palavras: é nisso que consiste a Tokenização

### Tokenização

A tokenização pode ser comparada a cortar um grande quebra-cabeça em peças menores. Na prática, trata-se de dividir o texto em unidades chamadas tokens, que podem ser palavras, subpalavras ou até mesmo caracteres individuais. Apesar de a ideia parecer simples, sua implementação pode ser bastante complexa.
Um dos primeiros pontos que precisamos ponderar é a forma como vamos dividir nosso texto já que isso influenciará diretamente o tamanho do nosso vocabulário e por consequência a eficiência do nosso modelo. Vamos usar um conjunto de textos em português como exemplo:

In [45]:
with open("../data/gigaverbo.txt", "r", encoding="utf-8") as f:
    text_raw = f.read()
    
print("Número de Caracteres:", len(text_raw))
print("Número de Palavras:", len(re.split(r"(\s)", text_raw)))
print("Número de Caracteres únicos:", len(set(text_raw)))
print("Número de Palavras únicas:", len(set(re.split(r"(\s)", text_raw))))

Número de Caracteres: 79444559
Número de Palavras: 25644299
Número de Caracteres únicos: 1265
Número de Palavras únicas: 748520


Avaliando dois cenários, onde em um quebramos o nosso texto por caractere (cada caractere é um id) e outro separando por palavra (cada palavra única é um id) vemos o impacto no tamanho do vocabulário: no primeiro cenário (separação por caractere) temos um vocabulário com 1265 ids e no segundo cenário (separação por palavras) temos um vocabulário com 748520 ids.

![](../assets/2-TOKENIZAÇÃO.png)

À primeira vista, a separação por caracteres pode parecer uma abordagem mais vantajosa, já que resulta em um vocabulário significativamente menor - mais de 500 vezes menor em comparação à divisão por palavras - , o que exige menos memória. No entanto, essa escolha traz consigo um impacto negativo, como podemos observar no exemplo abaixo:

In [46]:
phrase = "Hoje é um belo dia"

# Criando vocabulario de caracteres
vocab_char = sorted(set(text_raw))

# Criando vocabulario de palavras
vocab_word = sorted(set(re.split(r"(\s)", text_raw)))

print("Frase:", phrase)
print("Frase em caracteres:", [vocab_char.index(c) for c in phrase])
print("Frase em palavras:", [vocab_word.index(w) for w in re.split(r"(\s)", phrase)])

Frase: Hoje é um belo dia
Frase em caracteres: [43, 82, 77, 72, 3, 155, 3, 88, 80, 3, 69, 72, 79, 82, 3, 71, 76, 68]
Frase em palavras: [179435, 3, 746180, 3, 724737, 3, 355836, 3, 432683]


Podemos reparar que apesar do vocabulário por caracteres ser menor a conversão do conteúdo se torna muito mais extensa em comparação com o vocabulário por palavras. Isso quer dizer que a mesma frase necessitará de muito mais processamento para interpretação e relações entre esses caracteres pelo modelo, o que também não é desejado.

Um outro problema é o que fazer com palavras desconhecidas. Se tentarmos fazer o encode (converter texto para IDs) de uma frase como "Fui assistir o lançamento de um foguete" com nosso vocabulário, criado com texto de Machado de Assis, não teremos algumas palavras como "lançamento" e "foguete", ou então nomes próprios.

![](../assets/2-VOCABULARIO.png)

Para solucionarmos esses e outros problemas algumas soluções foram e são propostas rotineiramente. Recentemente, a Meta introduziu o Byte Latent Transformer (BLT), uma arquitetura que elimina a necessidade da tokenização. O BLT processa diretamente sequências de bytes brutos, agrupando-os dinamicamente em "patches" de tamanhos variados com base na complexidade dos dados.

Atualmente, a solução padrão ouro na industria, usada na maioria dos modelos comerciais, e que usaremos no nosso modelo, é a técnica de Byte Pair Encoding (BPE).

##  Byte Pair Encoding (BPE)


O **BPE** (e suas variações) é uma das técnicas mais usadas para lidar com a tokenização atualmente. Originalmente criado como um algoritmo de compressão em 1994, foi adaptado em 2015 por Sennrich et al. ([https://arxiv.org/abs/1508.07909](https://arxiv.org/abs/1508.07909)) para o processamento de linguagem natural para resolver, principalmente, o problema das palavras raras ou completamente desconhecidas.

Sua solução é relativamente simples mas eficás: encontrar um equilíbrio entre caracteres individuais e palavras completas, criando subpalavras que maximizam a eficiência.

Imagine que você está tokenizando a palavra “araraquara” usando BPE. No início, o algoritmo simplesmente divide a palavra em seus caracteres individuais:

`['a', 'r', 'a', 'r', 'a', 'q', 'u', 'a', 'r', 'a']`

A partir daí, começa a analisar o texto como um todo e identificamos os **pares de caracteres mais frequentes**. Nesse caso, o par “ar” é o mais comum. Então, ele o combina em um único token:

`['ar', 'ar', 'a', 'q', 'u', 'ar', 'a']`

O processo continua. O próximo par mais frequente, “ara”, também é combinado:

`['ar', 'ara', 'q', 'u', 'ara']`

No final, o que começou como 10 tokens foi reduzido para apenas 5. É esse tipo de compactação inteligente que torna o BPE tão eficiente. Ele identifica padrões frequentes e cria novos tokens que representam essas combinações, o que economiza espaço e melhora o desempenho do modelo.

A grande sacada do BPE é sua capacidade de representar palavras raras como combinações de subpalavras comuns. Em vez de simplesmente ignorar essas palavras ou tratá-las como completamente desconhecidas, o BPE permite que elas sejam representadas por combinações de tokens que já existem.

Por exemplo, uma palavra rara como “descentralização” pode ser dividida em subpalavras como “des”, “central” e “ização”. Mesmo que o modelo nunca tenha visto “descentralização” antes, ele ainda pode entender parte de seu significado por reconhecer essas subpartes.

Essa **flexibilidade** permite reduzir significativamente o tamanho do vocabulário, o que economiza recursos computacionais e facilita o treinamento dos modelos.

O número de fusões realizadas no Byte Pair Encoding (BPE) é definido por um hiperparâmetro arbitrário, esse número determina o tamanho do vocabulário final e, consequentemente, a granularidade dos tokens. Um vocabulário muito pequeno pode levar a uma divisão excessiva em tokens menores, enquanto um vocabulário muito grande pode resultar em um modelo menos eficiente em termos computacionais.

Além disso, a capacidade do tokenizador de representar palavras de forma eficiente depende fortemente do conjunto de textos utilizado. Um conjunto diversificado e representativo oferece ao BPE dados suficientes para aprender padrões frequentes e linguísticamente significativos. Por outro lado, um corpus limitado ou enviesado pode levar o tokenizador a priorizar combinações que não refletem bem a estrutura linguística do idioma ou do domínio.

# Nem tudo são flores...


A tokenização, embora essencial para o funcionamento de modelos de linguagem de grande escala (LLMs), carrega consigo uma série de desafios que justificam o título de "mal necessário".

#### A Fragmentação e o Contexto Perdido


Um dos maiores problemas da tokenização é a necessidade de fragmentação do texto em unidades menores, como palavras ou subpalavras. Essa divisão, enquanto necessária para o processamento por redes neurais, pode descontextualizar certos elementos linguísticos. Um exemplo clássico são os número, como por exemplo "6773", que, dependendo do tokenizador, pode ser dividido em dois ou mais tokens, dificultando a capacidade do modelo de lidar com cálculos ou de entender o significado numérico como um todo​

Esse problema se estende a palavras compostas, gírias e outros elementos linguísticos que, quando quebrados, perdem sua semântica original. A incapacidade de capturar adequadamente esses significados pode levar a previsões imprecisas e dificuldades em tarefas como tradução automática ou compreensão de linguagem natural.

#### O Impacto do Vocabulário e da Língua


Outro desafio está relacionado ao tamanho do vocabulário e à diversidade linguística. Tokenizadores treinados majoritariamente em textos em inglês, por exemplo, tendem a representar frases nesse idioma com menos tokens em comparação a outras línguas, como o português. Isso ocorre porque o treinamento com dados desbalanceados privilegia a compressão de padrões mais frequentes no idioma dominante​ no texto

Além disso, vocabulários maiores permitem representar mais informações com menos tokens, mas isso tem um custo: modelos com grandes vocabulários demandam mais memória e poder computacional, além de enfrentarem problemas de subtreinamento em tokens raros, que aparecem com pouca frequência nos dados​

#### A Complexidade Matemática e Computacional


A tokenização tem um impacto direto no desempenho computacional. Quando o texto é mais fragmentado, o número de tokens a serem processados aumenta, o que eleva tanto o tempo de treinamento quanto os requisitos de memória. Por outro lado, técnicas como o Byte Pair Encoding (BPE) compactam as informações em menos tokens, criando vocabulários mais densos.

Embora esses vocabulários densos sejam eficientes, eles exigem maior capacidade computacional para gerar embeddings mais complexos. Essa complexidade adicional é um aspecto importante a se considerar no desenvolvimento de modelos de linguagem. No próximo artigo, abordaremos mais detalhadamente o tema dos embeddings.

# Iniciando nosso LLM


Para iniciar o desenvolvimento do nosso LLM, o primeiro passo é construir o Tokenizador. Além de escolhermos a técnica que utilizaremos, que será a BPE (Byte Pair Encoding), é crucial selecionar o corpus adequado.

O conjunto de textos escolhido é de extrema importância, pois determinará os tokens que nosso modelo será capaz de reconhecer. Por exemplo, se o corpus não contiver a palavra "foguete", essa palavra será representada por vários tokens, dificultando a capacidade do nosso LLM de compreender seu significado de forma eficaz.

Nos treinaremos, tanto nosso tokenizador quanto nosso LLM, em uma parte do dataset chamado **GigaVerbo**. Esse dataset está disponível no [HuggingFace](https://huggingface.co/datasets/TucanoBR/GigaVerbo) e contém mais de 200 bilhões de tokens em português e faz parte do trabalho realizado pelo Nicholas Kluge na criação do modelo Tucano (o paper pode ser lido no Arxiv: [https://arxiv.org/abs/2411.07854](https://arxiv.org/abs/2411.07854))

Para conseguir acesso aos dados, usaremos o link disponibilizado pelo HuggingFace. Faremos o download do arquivo `train-00000-of-01573.parquet`


In [5]:
# Download primeira parte do dataset
df = pl.read_parquet('hf://datasets/TucanoBR/GigaVerbo/data/train-00000-of-01573.parquet')

print(f"{df.shape[0]} linhas e {df.shape[1]} colunas ({", ".join(df.columns)})")

92372 linhas e 4 colunas (text, label, probs, metadata)


Como podemos ver, o dataset contém mais de 92 mil textos (de diversos tamanhos) e todos recebem uma label, a probabilidade do texto pertencer ao label e a origem do conteúdo (metadata). O label indica se o texto foi considerado de boa qualidade ou não.

Para fazer essa classificação de qualidade o time do Tucano treinou um outro modelo para isso e pode ser visto em detalhes no paper. Para nós o importante aqui é entender que label 1 quer dizer que o texto foi considerado de qualidade e serão esses que usaremos no nosso treinamento.

In [8]:
# Filtrar apenas textos com label 1 (texto de qualidade) e concatenar em uma única string
text_raw = "<|eos|>".join(
    df.filter(
        pl.col('label') == 1)
    .select(
        pl.col('text'))
    .to_series()
    .to_list()
)

with open('../data/gigaverbo.txt', 'w') as f:
    f.write(text_raw)

Pegamos todos os conteúdos classificados com label 1 (texto de qualidade) e os concatenamos, separando-os por um token especial: <|eos|>.  

Os *special tokens* (tokens especiais) desempenham um papel crucial na estruturação e compreensão de dados textuais por modelos de linguagem. Esses tokens são elementos simbólicos adicionados à tokenização com funções específicas. Por exemplo, o token <|eos|> (end of sequence) é utilizado para indicar o fim de um segmento de texto, ajudando o modelo a diferenciar entre contextos ou trechos independentes.

## Treinando nosso Tokenizador


Para treinar o Tokenizador usaremos a biblioteca `sentencepiece` desenvolvida pelo google e que permite a criação de vocabulários a partir de diversas técnicas.

In [18]:
VOCAB_SIZE = 4096
SPECIAL_TOKENS = ["<|eos|>"]

spm.SentencePieceTrainer.train(
    input="../data/gigaverbo.txt",              # Caminho para o arquivo de contendo o conteúdo para treinar o tokenizador
    model_prefix="../models/gigaverbo_tk",     # Prefixo do nome do arquivo de saída
    vocab_size=VOCAB_SIZE,                          # Tamanho do vocabulário
    model_type="bpe",                               # Tipo do tokenizador
    self_test_sample_size=0,                        # Tamanho do conjunto de teste (não utilizaremos)
    character_coverage=0.995,                       # Fração de caracteres que devem ser cobertos pelo modelo
    input_format="text",                            # Formato do arquivo de entrada
    num_threads=os.cpu_count(),                     # Número de threads a serem utilizadas
    split_digits=True,                              # Separar os dígitos tratando-os como tokens separados
    allow_whitespace_only_pieces=True,              # Permitir tokens que representem apenas espaços em branco
    byte_fallback=True,                             # Utilizar tokenização de bytes como fallback
    unk_surface="<|unk|>",                          # Representação do token desconhecido
    normalization_rule_name="identity",             # Regra de normalização (identity = sem normalização)
    user_defined_symbols=SPECIAL_TOKENS             # Adiciona tokens especiais definidos pelo usuário
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../data/gigaverbo.txt
  input_format: text
  model_prefix: ../models/gigaverbo_tk
  model_type: BPE
  vocab_size: 4096
  self_test_sample_size: 0
  character_coverage: 0.995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  user_defined_symbols: <|eos|>
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_

er_interface.cc(425) LOG(INFO) Adding meta_piece: <0x7F>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x80>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x81>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x82>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x83>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x84>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x85>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x86>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x87>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x88>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x89>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8A>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8B>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8C>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8D>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8E>
trainer_inter

Após o treinamento, dois arquivos serão gerados: o `gigaverbo_tk.model` e o `gigaverbo_tk.vocab` . O arquivo .model contém as informações sobre o modelo, como tipo do tokenizador, forma em que números foram divididos entre outras informações. Já o .vocab é nossa grande lista contendo os tokens.

In [34]:
phrase = "Olá Mundo, tudo bem?<|eos|>"

tk = spm.SentencePieceProcessor(model_file="../models/gigaverbo_tk.model") # Instanciando o tokenizador treinado

phrase_encode = tk.encode(phrase)
phrase_tokens = tk.encode_as_pieces(phrase)
phrase_decode = tk.decode(phrase_encode)

print("Frase Tokenizada:", phrase_tokens)
print("Frase tokenizada:", phrase_encode)
print("Frase reconstruída:", phrase_decode)

Frase Tokenizada: ['▁Ol', 'á', '▁Mu', 'ndo', ',', '▁tudo', '▁bem', '?', '<|eos|>']
Frase tokenizada: [3188, 4047, 2362, 341, 4040, 1927, 824, 4091, 3]
Frase reconstruída: Olá Mundo, tudo bem?<|eos|>


No exemplo acima podemos ver a codificação da frase "Olá Mundo, tudo bem?" onde temos a palavra *Olá* sendo separada em dois tokens, `_Ol` (O sinal de _ significa que é um token inicial, ou seja, está no começo de uma palavra) e `á`. Podemos também ver que `_tudo` é um único token de ID **1927** enquanto o ? é representado pelo ID **4091**. Nosso token especial, <|eos|> é de ID **3**.

Vamos agora tokenizar nosso corpus (o conjunto de textos que extraímos do GigaVerbo) para que possamos utiliza-lo sem precisar converte-lo em tokens toda vez que formos iniciar o treinamento do LLM.

In [22]:
text_tokenized = tk.encode(text_raw)

# Salvando o texto tokenizado
with open("../data/gigaverbo_tokenized.txt", "w", encoding="utf-8") as f:
    f.write(" ".join([str(t) for t in text_tokenized]))

print("Amostra do texto tokenizado:", text_tokenized[:20])

Amostra do texto tokenizado: [2762, 1993, 596, 900, 271, 613, 445, 2710, 845, 4002, 372, 3651, 3479, 4091, 14, 14, 2385, 2258, 2188, 269, 1041, 1332, 4040, 2001, 1110, 374, 310, 1456, 739, 3763, 261, 866, 513, 301, 486, 547, 4037, 951, 1556, 296, 1294, 1064, 301, 888, 261, 1045, 1087, 4040, 323, 435, 298, 4036, 614, 302, 607, 1354, 596, 900, 271, 613, 733, 4030, 914, 445, 2710, 845, 4002, 372, 3651, 3479, 4040, 595, 323, 935, 1071, 1146, 2443, 362, 262, 1401, 297, 451, 269, 765, 4037, 14, 14, 4055, 4031, 885, 4040, 367, 2710, 845, 4002, 1339, 453, 1803, 828, 769]


Pronto! Nosso corpus está devidamente tokenizado. Agora, o próximo passo é transformar esses identificadores de tokens em algo que vá além de números aleatórios — algo que carregue significado e seja capaz de representar a semântica de cada token. É nesse momento que os ***Embeddings*** entram em cena e brilham!

# Conclusão


A tokenização é a ponte que conecta texto a números, mas é uma ponte cheia de buracos e obstáculos. Ela define como os modelos “enxergam” o mundo, mas também podem limitar sua capacidade de entender certos contextos. Apesar de seus problemas a tokenização ainda é indispensável.

Ao explorarmos as complexidades da tokenização, percebemos uma dança entre precisão semântica e eficiência computacional. A escolha do corpus e técnica de tokenização, assim como o tamanho do vocabulário são passos importantíssimos no caminho para desenvolver um LLM.

No próximo artigo, exploraremos outra peça fundamental no funcionamento dos grandes modelos de linguagem: os **embeddings**. Se o mecanismo de atenção é o coração dos LLMs, os embeddings são o sangue que flui, carregando os significados e relações complexas que sustentam toda a compreensão linguística do modelo.